<a href="https://colab.research.google.com/github/PASTANERD/HGU_PYNQ_Project/blob/master/hw7_21400646.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align:right"> <h1> Machine Learning</h1> <h3> HW #7 </h3> <h4> 21400646 Lim Chae Eon </h4> </div>

### Final Report: BNN(Binarized Neural Network)

1. **Introduction**
  1. BNN을 선택한 이유 현재 연구하고 있는 분야
  1. BNN 요약
1. **Algorithms and Implements BNN**
  1. Parameter Binarization
  1. Gradient Propagation in Binarized Neural Network
  1. Binarized Fully Connected Layer
  1. Binarized Neural Network
  1. Training Code
  1. Testing Code
  1. Configuration and Datatsets
1. **Reuslt(Comparing Accuracy)**
  1. Training and Testing MNIST Datasets on Binarized Neural Network 
  1. Training and Testing MNIST Datasets on Deep Neural Network 
  1. Comparing the results
1. **Appendix**
1. **References**




### 1\. Introduction
#### **BNN: Binarized Neural Network**
>Binarized Neural Network는 Neural Network의 Weight(가중치)와 Activation(Activation function의 결과)가 Run-time 중에 bit로 구성되어있는 딥러닝 네트워크를 의미한다[1].

딥러닝 네트워크를 이진화(Binarization)하는 방법은, Training이 끝난 후 결과로 얻은 Parameter를 이진화하거나, Training 과정에서 Propagation 중에 이진화를 처리하는 방법이 있다. 알려진 바로는, Training 과정에서 이진화를 하는 것이 더 좋은 결과를 얻는다고 한다. 부가적으로, Training 시 Binary Weight와 Binary Activation은 Parameter의 Gradient를 구하는데 사용할 수 있다.
</br></br>
#### **FPGA Accelerator for Deep Learning**
FPGA는 일반적으로 ASIC 설계의 검증 및 테스트용으로 많이 사용되어왔으나, 최근 딥러닝의 발전과 함께 가속기로써 잠재된 영향력이 대두되고 있다. FPGA는 현장에서 재구성 가능한 하드웨어를 일컫는 말로, CPU나 GPU처럼 기능이 정해지지 않고, 프로그래밍으로 하드웨어의 기능을 조절할 수 있다. FPGA를 이용하여 가속기를 설계하면 GPU처럼 Deep Learning 연산을 가속시키는 것이 가능하다. 또한 GPU에 비해 훨씬 더 적은 전력을 소모하면서 GPU에 못지 않은 성능을 보여주고 있다[2].
</br></br>
#### **BNN and FPGA Accelerator on Embedded System**
대부분의 컴퓨팅 하드웨어들은 기본적으로 논리 연산을 바탕으로 하고 있다. 그러나 Deep Learning에서 가장 많은 연산을 차지하는 Linear 연산의 경우 대부분 Parameter들이 32-bit Floating Point 로 처리된다. 부동소수점(Floating Point) 연산은 많은 메모리를 필요로 하며 처리될 데이터의 크기가 커짐으로써 연산의 부하를 높이게 된다. BNN은 Parameter들을 1-bit로 표현함으로써, 메모리 사용을 크게 줄이고, 동시에 Matrix Multiplication 같은 복잡한 연산을 Bitwise Operation으로 대체할 수 있게 한다. 이는 하드웨어의 퍼포먼스를 극대화하는 것으로 그치지 않고, FPGA 가속기로 하여금 리소스와 전력이 제한된 Embedded System에서 Deep Learning 연산을 가능하게 한다.
</br></br>
#### **보고서를 시작하며,**
본 보고서는 이강교수님의 FPGA 가속기 연구실에서 **임베디드 환경에서 딥러닝 네트워크 추론을 위한 FPGA 가속기 설계** 연구에서 사용되는 BNN을 구현하고 이를 검증하는 것을 담았다. 저자가 속한 연구실은 연구에서 BNN을 이용하지만, BNN을 직접 연구하지 않기 때문에 이를 구현하는 과정은 보고서에서 처음 이루어졌다. 또한 연구에서 사용하던 BNN은 Theano, Lasagne 프레임워크 기반으로 구현되어 있었다. 따라서 이 보고서를 통해 PyTorch 기반에서 BNN을 새롭게 구현하고자 하였다.



### 2\. Algorithms and Implements BNN

#### 2.1. Parameter Binarization
BNN에서 이진화 되는 Parameter는 Weight와 Activation이 있다. 기본적으로 이진화 법칙은 아래 식을 따른다[1].

| Deterministic Binarization | Stochastic Binarization |
| :---: | :---: |
| $$x^b = Sign(x) =   \begin{cases} +1  & \quad \text{if } x \geq 0 \\
    -1  & \quad \text{otherwise}
  \end{cases}$$ | $$ 
x^b =   \begin{cases}
    +1  & \quad \text{with probability } p =  \sigma(x) \\
    -1  & \quad \text{with probability } 1 - p
  \end{cases} 
$$ |

</br>

Stochastic Binarization에서 $Sign(x)$는 입력 x의 부호를 구하는 함수 $\text{(Signum Function)}$이며, $\sigma(x)는 \text{ Hard Sigmoid function}$를 뜻하며, 정의는 아래와 같다.</br>
>$
  \sigma(x) = clip(\frac{x+1}{2} ,0, 1) = max(0, min(1, \frac{x+1}{2}))
$

위 식에서 알 수 있듯이, Binary라고 하면 기본적인 표현법은 0 과 1이지만, 식에서 표현은 +1과 -1로 구성하고 있다. 이와 같이 표현하는 이유는, 컴퓨터에서 사용하는 데이터의 범위 표현 방법을 그대로 차용하기 때문이다. 예를 들어, C언어에서 int와 같은 경우 기본적으로 32bit로 구성되어 있으며, 부호를 표현하기 위해 $[-2^{15},~ ... ~, 2^{15}-1]$ 사이의 숫자를 표현할 수 있다. 마찬가지로 1bit의 Binary 표현법은 0과 1이지만, 컴퓨터 부호를 포함하여 표현하면 1과 -1로 볼 수 있다. 동시에 0과 1처럼 숫자의 범위가 2개 밖에 없기 때문에 연산 방식을 기존과 다르게 Bit operation을 적용할 수 있다.

아래는, 앞서 설명한 이진화를 코드로 구현한 것이다.

In [0]:
def HardSigmoid(tensor):
    return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)

def Binarize(tensor,quant_mode='det'):
  # Deterministic Method
    if quant_mode=='det':
        return tensor.sign()
  # Stochastic Method        
    else:
        return HardSigmoid(tensor)


#### 2.2. Gradients Propagation in Binary
앞서 Deterministic Binarization에서, Parameter를 Sign($ x^b = Sign(x) $)를 이용하여 계산했다. Binary Parameter의 Gradient Propagation은 아래 식과 같이 표현할 수 있다[1]. 

>$
 g_x = g_{x^b} * 1_{|x|\leq1} \\ 
$

이 때, $1_{|x|\leq1}$의 의미는 다음과 같다.

>$
1_{|x|\leq1} = \begin{cases}
    1  & \quad \text{if } -1 \leq x \leq 1 \\
    0  & \quad \text{otherwise}
  \end{cases}  
$

이는 _Hard Tanh_ 연산으로 표현할 수 있다.</br>
![activation2](https://pytorch.org/docs/stable/_images/Hardtanh.png)


이는 Hinton의 STE(Straight-Through Estimator)의 아이디어를 이용한 것이다. 즉, Sign 함수를 사용하게 되면, Sign 함수를 미분한 결과는 Delta 함수처럼 나타나기 때문에, Parameter 값이 조금이라도 크거나 작을 경우, Gradient가 Saturation 되는 경향이 쉽게 발생한다. 따라서 Parameter들이 |1| 보다 클 경우 이를 차단하는 방식을 사용하였다.
(이는 Stochastic 에서도 동일하게 적용할 수 있다)

#### 2.3. Binarized Fully Connected Layer

Binarization을 사용하여 딥러닝 네트워크를 만들기 위해서, 앞서 정의한 Binarize() 함수를 사용하는 Fully Connected Layer와 2-Dimension Convolutional Layer를 새롭게 정의하였다.

In [0]:
import torch
import torch.nn as nn

class BinarizeFullyConnected(nn.Linear):
    def __init__(self, *kargs, **kwargs):
        super(BinarizeFullyConnected, self).__init__(*kargs, **kwargs)

    def forward(self, input):

        # Do binarization except first layer
        if input.size(1) != 784:  # 28 x 28 = 784
            input.data=Binarize(input.data)

        # Make orginal attribute in Parameter to save the original weight
        if not hasattr(self.weight,'orginal'):
            self.weight.orginal=self.weight.data.clone()
        self.weight.data=Binarize(self.weight.orginal)

        # Do Linear operation with binarized weights
        out = nn.functional.linear(input, self.weight)

        # Add a bias to the result 
        if not self.bias is None:
            self.bias.orginal=self.bias.data.clone()
            out += self.bias.view(1, -1).expand_as(out)

        return out

#### 2.4. Binarized Neural Network

MNIST 데이터셋을 Training 하기 위한 네트워크를 구성하였다. 기본적으로 4개의 Fully Connected Layer를 나열하였으며, 각 레이어마다 Activation Function으로 HardTanh()를 사용하였다. 추가로 네트워크의 정확도를 위해 Batch Normalization을 적용하였다.

In [0]:
class BinaryNet(nn.Module):
  def __init__(self, binary=True, factor = 1):
    self.binarized = binary
    # factor means scale factor to increase the size of network easily or dynamically
    # Binarized fully connected configuration
    if binary:
      super(BinaryNet, self).__init__()
      self.fc1 = BinarizeFullyConnected(28*28, 1024*factor)
      self.fc2 = BinarizeFullyConnected(1024*factor, 1024*factor)
      self.fc3 = BinarizeFullyConnected(1024*factor, 1024*factor)
      self.fc4 = BinarizeFullyConnected(1024*factor, 10)
    # Normal Fully connected configuration
    else:
      super(BinaryNet, self).__init__()
      self.fc1 = nn.Linear(28*28, 1024*factor)
      self.fc2 = nn.Linear(1024*factor, 1024*factor)
      self.fc3 = nn.Linear(1024*factor, 1024*factor)
      self.fc4 = nn.Linear(1024*factor, 10)
      
    # Set other layers (activation, batch normalization, softmax)
    self.htanh1 = nn.Hardtanh()               # Activate function with HardTanh in PyTorch Library. Default min and max are -1 and 1.
    self.bn1 = nn.BatchNorm1d(1024*factor)
    self.htanh2 = nn.Hardtanh()
    self.bn2 = nn.BatchNorm1d(1024*factor)
    self.htanh3 = nn.Hardtanh()
    self.bn3 = nn.BatchNorm1d(1024*factor)
    self.logsoftmax=nn.LogSoftmax()

  # Configuration for forward propagation
  def forward(self, input):
    x = input.view(-1, 28*28)
    x = self.fc1(x)
    x = self.bn1(x)
    x = self.htanh1(x)
    x = self.fc2(x)
    x = self.bn2(x)
    x = self.htanh2(x)
    x = self.fc3(x)
    x = self.bn3(x)
    x = self.htanh3(x)
    x = self.fc4(x) 
    x = self.logsoftmax(x)
    return x

#### 2.5. Training Code
Training 과정을 정의하기 위한 부분이다.

In [0]:
from torch.autograd import Variable

def train(model, optimizer, criterion, epoch):
  model.train()
  # train each mini batches
  for idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data.to(device)), Variable(target.to(device))
    output = model(data)
    loss = criterion(output, target)

    if epoch % 40 == 0:
      optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.1
    
    optimizer.zero_grad()
    loss.backward()
    for p in list(model.parameters()):
      if hasattr(p,'orginal'):
        p.data.copy_(p.orginal)
    optimizer.step()

    for p in list(model.parameters()):
      if hasattr(p,'orginal'):
        p.orginal.copy_(p.data.clamp_(-1,1))
    
    if idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item()))



#### 2.6. Testing Code
Testing 과정을 정의하기 위한 부분이다.

In [0]:
def save_state(model, acc):
    print('==> Saving model ...')
    state = {
            'acc': acc,
            'state_dict': model.state_dict(),
            }
    for key in state['state_dict'].keys():
        if 'module' in key:
            state['state_dict'][key.replace('module.', '')] = \
                    state['state_dict'].pop(key)
    # torch.save(state, 'models/'+args.arch+'.best.pth.tar')
    if model.binarized:
      model_name = "binary_"
    else:
      model_name = "normal_"
    torch.save(state, model_name+'best.pth.tar')

def test(model, criterion):
  global best_acc
  model.eval()
  test_loss = 0
  correct = 0
  
  with torch.no_grad():
    for data, target in test_loader:
      data, target = Variable(data.to(device)), Variable(target.to(device))
      output = model(data)
      test_loss += criterion(output, target).item()   # sum up batch loss
      pred = output.data.max(1, keepdim=True)[1]      # get the index of the max log-probability
      correct += pred.eq(target.data.view_as(pred)).cpu().sum()
  
    acc = 100. * float(correct) / len(test_loader.dataset)
    if (acc > best_acc):
        best_acc = acc
        # if not evaluate:
        save_state(model, best_acc)
  
  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. *correct / len(test_loader.dataset)))


#### 2.7. Configuration and Datasets

MNIST Training을 위해 MNIST 데이터셋을 다운로드하고, 디바이스 설정과 Training에 필요한 factor를 설정한다.

Loss Funciton으로는 Cross Entropy Function 을 사용하였고, Optimizer로는 Adam을 사용하였다.

|Name|Factor|
|:---:|:---:|
|Optimizer|__Adam__|
|Loss Funciton|__Cross Entropy__|
|epoch|100|
|batch size|64|
|learning rate|0.0001|

In [0]:
from __future__ import print_function
from torchvision import datasets, transforms
import torch.optim as optim

seed = 1                  # Parameter for Random Generator in PyTorch
epochs = 100              # Number of epochs
batch_size = 64           # Size of the batch in each training
test_batch_size = 1000    # Size of the batch in each test
lr = 1e-4                 # Learning Rate
interval = 10             # Term for printing the result of the training at each epoch
best_acc = 0.0            # Variable to save the highest accuracy
save_model = True

use_cuda = torch.cuda.is_available()  # use Cuda


device = torch.device("cuda" if use_cuda else "cpu")
print("Computing Environment: ", device)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
transform=transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((0.1307,), (0.3081,)) ])


# MNIST dataset loading
torch.manual_seed(seed)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('drive/My Drive/{8th Semester}/Machine Learning/hw/hw7/', train=True, 
                   download=True, transform=transform),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('drive/My Drive/{8th Semester}/Machine Learning/hw/hw7/', train=False, 
                   transform=transform),
    batch_size=test_batch_size, shuffle=True, **kwargs)

# make interval to print training processes and losses
log_interval = len(train_loader.dataset) / batch_size / interval

# scale factor of the linear layer
factor = 2

# make models, criterion, and optimizer
binarized_model = BinaryNet(binary=True, factor = factor).to(device)  # make binarized model 
normal_model = BinaryNet(binary=False, factor = factor).to(device)    # make normal model 
criterion = nn.CrossEntropyLoss()
b_optimizer = optim.Adam(binarized_model.parameters(), lr=lr)
n_optimizer = optim.Adam(normal_model.parameters(), lr=lr)

Computing Environment:  cuda



## 3\. Result(Comparing Accuracy)
#### 3.1. Training and Testing MNIST Datasets on Binarized Neural Network 




In [0]:
# Train and test on binarized model 
for epoch in range(1, epochs + 1):
  train(binarized_model, b_optimizer, criterion, epoch)
  test(binarized_model, criterion)
  if epoch % 40 == 0:
    b_optimizer.param_groups[0]['lr'] = b_optimizer.param_groups[0]['lr']*0.1


binary_best = best_acc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 79.009827
Train Epoch: 1 [24000/60000 (40%)]	Loss: 9.532460
Train Epoch: 1 [48000/60000 (80%)]	Loss: 6.324212
==> Saving model ...

Test set: Average Loss: 0.0049, Accuracy: 9184/10000 (91.84%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 4.093976
Train Epoch: 2 [24000/60000 (40%)]	Loss: 3.844896
Train Epoch: 2 [48000/60000 (80%)]	Loss: 5.748398
==> Saving model ...

Test set: Average Loss: 0.0033, Accuracy: 9415/10000 (94.15%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.343578
Train Epoch: 3 [24000/60000 (40%)]	Loss: 2.698038
Train Epoch: 3 [48000/60000 (80%)]	Loss: 3.001101
==> Saving model ...

Test set: Average Loss: 0.0036, Accuracy: 9437/10000 (94.37%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.905661
Train Epoch: 4 [24000/60000 (40%)]	Loss: 3.623110
Train Epoch: 4 [48000/60000 (80%)]	Loss: 0.010904
==> Saving model ...

Test set: Average Loss: 0.0031, Accuracy: 9497/10000 (94.97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.687773
Train Epoch: 5 [24000/60000 (40%)]

#### 3.2. Training and Testing on Deep Neural Network 

In [0]:
# Train and test on Normal model 
for epoch in range(1, epochs + 1):
  train(normal_model, n_optimizer, criterion, epoch)
  test(normal_model, criterion)
  if epoch % 40 == 0:
    n_optimizer.param_groups[0]['lr'] = n_optimizer.param_groups[0]['lr']*0.1

normal_best = best_acc

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.483741


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.283368
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.263458
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.483741


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.283368
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.263458

Test set: Average Loss: 0.0001, Accuracy: 9613/10000 (96.13%)


Test set: Average Loss: 0.0001, Accuracy: 9613/10000 (96.13%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.043968
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.043968
Train Epoch: 2 [24000/60000 (40%)]	Loss: 0.037649
Train Epoch: 2 [24000/60000 (40%)]	Loss: 0.037649
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.097217
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.097217

Test set: Average Loss: 0.0001, Accuracy: 9712/10000 (97.12%)


Test set: Average Loss: 0.0001, Accuracy: 9712/10000 (97.12%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.028836
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.028836
Train Epoch: 3 [24000/60000 (40%)]	Loss: 0.044312
Train Epoch: 3 [24000/60000 (40%)]	Loss: 0.044312
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.132647
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.132647

Test set: Average Loss: 0.0001, Accuracy: 9702/10000 (97.02%)



#### 3.3. Comparing the results

MNIST 데이터셋을 바탕으로 BNN을 트레이닝한 결과를 보았을 때 결과는 다음과 같았다. DNN(Deep Neural Network)에서 최대 정확도가 98.17%이고, BNN(Binarized Neural Network) 환경에서 최대 97.7%의 정확도를 가진 것을 확인할 수 있다(이 내용은 실험시 소폭 달라질 수 있음). DNN과 BNN의 정확도 차이가 약 0.47% 인 것으로 나타난 것을 볼 수 있다. 이는 Network의 Parameter를 이진화시킴으로써 크기를 크게 줄인 반면, 정확도는 소폭 감소하여 성능이 유지되는 것을 알 수 있다.

BNN Parameter를 출력하여 네트워크의 Parameter가 Binarized 됬는지 확인할 수 있다. 아래 출력된 결과를 통해 1과 -1이 weight의 값으로 사용된 것을 볼 수 있다.

In [0]:
import matplotlib.pylab as plt
import numpy as np

print("Binarized Nerual Netowork: ", binary_best,"%", "\nDeep Neural Network: ", normal_best, "%\n")
print("Example Parameters in BNN\n",binarized_model.fc2.weight)
print("\nExample Parameters in DNN\n", normal_model.fc2.weight)

Binarized Nerual Netowork:  97.7 % 
Deep Neural Network:  98.17 %

Example Parameters in BNN
 Parameter containing:
tensor([[-1., -1.,  1.,  ..., -1., -1., -1.],
        [-1.,  1.,  1.,  ...,  1., -1.,  1.],
        [-1., -1., -1.,  ..., -1.,  1.,  1.],
        ...,
        [ 1.,  1., -1.,  ...,  1., -1.,  1.],
        [-1.,  1., -1.,  ..., -1.,  1., -1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]], device='cuda:0',
       requires_grad=True)

Example Parameters in DNN
 Parameter containing:
tensor([[-1.0074e-03, -5.9346e-05, -2.5056e-02,  ..., -2.7849e-02,
          1.6668e-02, -6.0850e-03],
        [ 4.8277e-02,  1.7304e-02, -7.2783e-03,  ...,  2.7869e-02,
          3.1388e-02, -3.0012e-02],
        [-1.5052e-02, -5.4204e-03, -5.8841e-03,  ...,  4.3406e-02,
         -2.5258e-02,  9.0726e-03],
        ...,
        [-1.0635e-02,  1.1019e-02, -3.4251e-03,  ..., -2.3202e-02,
          6.0548e-03, -4.1337e-03],
        [ 1.4162e-03,  1.1785e-02, -2.1178e-02,  ...,  3.0263e-02,
        

## 4\. Appendix

보고서에서 구현한 내용을 가속기로 설계하여 실제 FPGA 보드에서 구현할 수 있다[2][3]. 다만 FPGA 가속기로 설계하는 과정은 본 과목인 머신 러닝과 크게 관련이 없는 관계로, 본 보고서에 기술하지 않았다.

![pynq-z2](https://media-cdn.seeedstudio.site/media/catalog/product/cache/9d0ce51a71ce6a79dfa2a98d65a0f0bd/0/3/03_pynq-z2.jpg)

부록에는 위에서 정의한 BNN 네트워크를 가속기로 구현하였을 때, 임베디드 FPGA 보드에서 딥러닝이 연산 성능을 테스트하는 노트북을 담았다. 아래 설명된 노트는 본 연구실에서 사용한 노트북을 참조한 것이다. 가속기로 사용한 보드는 TUL사에서 공급하는 PYNQ-Z2 보드이다. FPGA 칩으로는 Xilinx의 ZYNQ-7000SoC 가 내장되어있다.



### **BNN on PYNQ-Z2**

This notebook covers how to use Binary Neural Networks on Pynq[2]. 
It shows an example of handwritten digit recognition using a binarized neural network composed of 4 fully connected layers with 1024 neurons each, trained on the MNIST dataset of handwritten digits. 
In order to reproduce this notebook, you will need an external USB Camera connected to the PYNQ Board.

### 4.1. Import the package

```python
import bnn
```

### 4.2. Checking available parameters

By default the following trained parameters are available for LFC network using 1 bit for weights and 1 threshold for activation:


```python
print(bnn.available_params(bnn.NETWORK_LFCW1A1))
```

    ['mnist', 'chars_merged']


Two sets of weights are available for the LFCW1A1 network, the MNIST and one for character recognition (NIST).

### 4.3. Instantiate the classifier

Creating a classifier will automatically download the correct bitstream onto the device and load the weights trained on the specified dataset. This example works with the LFCW1A1 for inferring MNIST handwritten digits.
Passing a runtime attribute will allow to choose between hardware accelerated or pure software inference.


```python
hw_classifier = bnn.LfcClassifier(bnn.NETWORK_LFCW1A1,"mnist",bnn.RUNTIME_HW)
sw_classifier = bnn.LfcClassifier(bnn.NETWORK_LFCW1A1,"mnist",bnn.RUNTIME_SW)
```


```python
print(hw_classifier.classes)
```

    ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


### 4.4. Load the image from the camera

The image is captured from the external USB camera and stored locally. The image is then enhanced in contract and brightness to remove background noise. 
The resulting image should show the digit on a white background:

![field](https://trello-attachments.s3.amazonaws.com/5dafff41d7f4747dcbf310a1/5def2cb3fc678a823ccd45db/532f51b7f9843dad907bff8f01869ee4/IMG_4321.jpg)

```python
import cv2
from PIL import Image as PIL_Image
from PIL import ImageEnhance
from PIL import ImageOps

# says we capture an image from a webcam
cap = cv2.VideoCapture(0) 
_ , cv2_im = cap.read(0)
i = 0
while(_ == False):
    _ , cv2_im = cap.read(0)
    i += 1
    if(i >= 100):
        i = 0
        print(_)
cv2_im = cv2.cvtColor(cv2_im,cv2.COLOR_BGR2RGB)
img = PIL_Image.fromarray(cv2_im).convert("L") 
#Image enhancement                
contr = ImageEnhance.Contrast(img)
img = contr.enhance(3)                                                    # The enhancement values (contrast and brightness) 
bright = ImageEnhance.Brightness(img)                                     # depends on backgroud, external lights etc
img = bright.enhance(4.0)          
                                            # Rotate the image (depending on camera orientation)
#Adding a border for future cropping
img = ImageOps.expand(img,border=80,fill='white') 
img

```
![png](https://trello-attachments.s3.amazonaws.com/5dafff41d7f4747dcbf310a1/5def2cb3fc678a823ccd45db/a87873ce17d41147dfbd3823821d44dc/output_9_0.png)
```python
width = img.size[0]
height = img.size[1]

m_width = 250
m_height = int(height*(m_width/width))

w_diff = int((width - m_width)/2)
h_diff = int((height - m_height)/2)

data = img.crop((w_diff,h_diff,w_diff+m_width,h_diff+m_height))
print(data.size)
data
```

    (250, 200)

![png](https://trello-attachments.s3.amazonaws.com/5dafff41d7f4747dcbf310a1/5def2cb3fc678a823ccd45db/8c5e22980d3ac7786807c484731c07b4/output_10_1.png)



### 4.5. Crop and scale the image

The center of mass of the image is evaluated to properly crop the image and extract the written digit only. 


```python
from PIL import Image as PIL_Image
import numpy as np
import math
from scipy import misc

#Find bounding box  
# 테두리 잘라서 숫자만 남기는 부분
inverted = ImageOps.invert(data)  
box = inverted.getbbox()
img_new = data.crop(box)

#Resize and add padding blacks
sqr_size = 18
width, height = img_new.size  
ratio = min((sqr_size/height), (sqr_size/width))  
background = PIL_Image.new('RGB', (sqr_size,sqr_size), (255,255,255))  
if(height == width):  
    img_new = img_new.resize((sqr_size,sqr_size))  
elif(height>width):  
    img_new = img_new.resize((int(width*ratio),sqr_size))  
    background.paste(img_new, (int((sqr_size-img_new.size[0])/2),int((sqr_size-img_new.size[1])/2)))  
else:  
    img_new = img_new.resize((sqr_size, int(height*ratio)))  
    background.paste(img_new, (int((sqr_size-img_new.size[0])/2),int((sqr_size-img_new.size[1])/2)))  
  

background  
img_data=np.asarray(background)  
img_data = img_data[:,:,0]  



img_new
```




![png](https://trello-attachments.s3.amazonaws.com/5dafff41d7f4747dcbf310a1/5def2cb3fc678a823ccd45db/5210fbb92784d8337a92fd23247f2f35/output_12_0.png)




```python
padd_img = np.zeros((28,28))

diff = int((28 - sqr_size) / 2)

for i in range(0,28):
    if i < diff:
        padd_img[i] = [255 for i in range(0,28)]
    elif i > 27-diff:
        padd_img[i] = [255 for i in range(0,28)]
    else:
        temp = [255 for i in range(diff)]
        temp.extend(img_data[i-diff])
        temp.extend([255 for i in range(diff)])
        padd_img[i] = temp
padd_img.shape

misc.imsave('/home/xilinx/img_webcam_mnist.png', padd_img) 
```

### 4.6. Convert to BNN input format

The image is resized to comply with the MNIST standard. The image is resized at 28x28 pixels and the colors inverted. 


```python
from array import *
from PIL import Image as PIL_Image
from PIL import ImageOps
img_load = PIL_Image.open('/home/xilinx/img_webcam_mnist.png').convert("L")  
# Convert to BNN input format  
# The image is resized to comply with the MNIST standard. The image is resized at 28x28 pixels and the colors inverted.   
  
#Resize the image and invert it (white on black)  흑백 반전
smallimg = ImageOps.invert(img_load)  
smallimg = smallimg.rotate(0)  
  
data_image = array('B')  
  
pixel = smallimg.load()  
for x in range(0,28):  
    for y in range(0,28):  
        if(pixel[y,x] == 255):  
            data_image.append(255)  
        else:  
            data_image.append(1)  
          
# Setting up the header of the MNIST format file - Required as the hardware is designed for MNIST dataset         
hexval = "{0:#0{1}x}".format(1,6)  
header = array('B')  
header.extend([0,0,8,1,0,0])  
header.append(int('0x'+hexval[2:][:2],16))  
header.append(int('0x'+hexval[2:][2:],16))  
header.extend([0,0,0,28,0,0,0,28])  
header[3] = 3 # Changing MSB for image data (0x00000803)  
data_image = header + data_image  
output_file = open('/home/xilinx/img_webcam_mnist_processed', 'wb')  
data_image.tofile(output_file)  
output_file.close()   
smallimg
```




![png](https://trello-attachments.s3.amazonaws.com/5dafff41d7f4747dcbf310a1/5def2cb3fc678a823ccd45db/91256c2ba4a08d542fe730fa53436137/output_15_0.png)



### 4.7. Launching BNN in hardware

The image is passed in the PL and the inference is performed. Use `classify_mnist` to classify a single mnist formatted picture.


```python
class_out = hw_classifier.classify_mnist("/home/xilinx/img_webcam_mnist_processed")
print("Class number: {0}".format(class_out))
print("Class name: {0}".format(hw_classifier.class_name(class_out)))
```

    Inference took 24.00 microseconds
    Classification rate: 41666.67 images per second
    Class number: 4
    Class name: 4


### 4.8. Launching BNN in software

The inference on the same image is performed in sofware on the ARM core


```python
class_out=sw_classifier.classify_mnist("/home/xilinx/img_webcam_mnist_processed")
print("Class number: {0}".format(class_out))
print("Class name: {0}".format(hw_classifier.class_name(class_out)))
```

    Inference took 84201.00 microseconds
    Classification rate: 11.88 images per second
    Class number: 4
    Class name: 4


### 4.9. Reset the device


```python
from pynq import Xlnk

xlnk = Xlnk()
xlnk.xlnk_reset()
```



## 5\. References

>[1] Courbariaux, M., Hubara, I., Soudry, D., El-Yaniv, R., & Bengio, Y. (2016). "Binarized neural networks: Training deep neural networks with weights and activations constrained to +1 or -1". arXiv preprint arXiv:1602.02830.
>
>[2] Umuroglu, Y., Fraser, N. J., Gambardella, G., Blott, M., Leong, P., Jahre, M., & Vissers, K. (2017, February). "Finn: A framework for fast, scalable binarized neural network inference". In Proceedings of the 2017 ACM/SIGDA International Symposium on Field-Programmable Gate Arrays (pp. 65-74). ACM.
>
>[3] Chae Eon Lim, Geun Su Song, Yun Hye Park., Yoon Seong Lim, Kang Yi, “FPGA Accelerator Design for Handwritten Digit Recognition in Embedded System”, 2019 Fall Conference on Korea Multimedia Society

